# Named Entity Recognition with Bidirectional LSTMs
- TUTORIAL: [Sequence Models and Long Short-Term Memory Networks](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#sequence-models-and-long-short-term-memory-networks)

In [1]:
# !pip install torcheval

In [19]:
# imports
# import pdb # for step by step debugging
# assert(False) # use to stop at a specific line (think of like stop, quit, exit, etc)
import gzip
import torch
import shutil


import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


from tqdm import tqdm
from datasets import Dataset


from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils import clip_grad_norm_
from sklearn.model_selection import train_test_split
from torcheval.metrics.functional import multiclass_f1_score


/Users/brinkley97/opt/anaconda3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Data

In [3]:
# Load and Update Data
def load_data(file_path, new_sentence_col_name, word_col_name, ner_tag_col_name):
    sentence_idxs = []
    sentences = []
    sentences_ner_tags = []

    words = []
    ner_tags = []

    space_sentence_idxs = []
    space_words = []
    space_ner_tags = []
    
    with open(file_path, 'r') as f:
        lines = f.read().split("\n") # separate each sentence by new line
        # print(lines)
    sentence_idx = 0
    for line in lines:
        # text_idx, input_text, text_ner_tag = line.split("[")
        
        each_line = line.split(" ")
        # print(len(each_line))
        if len(each_line) != 1:
            sentence_idx += 1
            text_idx, input_text, ner_tag = each_line[0], each_line[1], each_line[2]
            # print(f"Line {text_idx} has word {input_text}, with NER Tag {ner_tag} --- {sentence_idx}")
            sentence_idxs.append(sentence_idx)

            words.append(input_text)
            ner_tags.append(ner_tag)
            

            space_sentence_idxs.append(sentence_idx)
            space_words.append(input_text)
            space_ner_tags.append(ner_tag)
        else:
            

            if words and ner_tags:
                
                # print(f"words {words}")
                sentences.append(words)
                # print(f"ner_tags {ner_tags}")
                sentences_ner_tags.append(ner_tags)
                
                words = []
                ner_tags = []

            sentence_idx = 0
            reset_sentence_idx, input_text, ner_tag = " ", " ", " "
            space_sentence_idxs.append(reset_sentence_idx)
            space_words.append(input_text)
            space_ner_tags.append(ner_tag)

    # print(sentences)
    df = pd.DataFrame(zip(sentence_idxs, words, ner_tags), columns=[new_sentence_col_name, word_col_name, ner_tag_col_name])
    with_space_df = pd.DataFrame(zip(space_sentence_idxs, space_words, space_ner_tags), columns=[new_sentence_col_name, word_col_name, ner_tag_col_name])


    return df, with_space_df, sentences, sentences_ner_tags

In [4]:
train_file_path = "data/train"
new_sentence_col_name = 'New Sentence Index'
word_col_name = 'Word'
ner_tag_col_name = 'NER Tag'

df, with_space_df, sentences, sentences_ner_tags = load_data(train_file_path, 'new_sentence_col_name', 'word_col_name', 'ner_tag_col_name')

In [5]:
sentences

[['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'],
 ['Peter', 'Blackburn'],
 ['BRUSSELS', '1996-08-22'],
 ['The',
  'European',
  'Commission',
  'said',
  'on',
  'Thursday',
  'it',
  'disagreed',
  'with',
  'German',
  'advice',
  'to',
  'consumers',
  'to',
  'shun',
  'British',
  'lamb',
  'until',
  'scientists',
  'determine',
  'whether',
  'mad',
  'cow',
  'disease',
  'can',
  'be',
  'transmitted',
  'to',
  'sheep',
  '.'],
 ['Germany',
  "'s",
  'representative',
  'to',
  'the',
  'European',
  'Union',
  "'s",
  'veterinary',
  'committee',
  'Werner',
  'Zwingmann',
  'said',
  'on',
  'Wednesday',
  'consumers',
  'should',
  'buy',
  'sheepmeat',
  'from',
  'countries',
  'other',
  'than',
  'Britain',
  'until',
  'the',
  'scientific',
  'advice',
  'was',
  'clearer',
  '.'],
 ['"',
  'We',
  'do',
  "n't",
  'support',
  'any',
  'such',
  'recommendation',
  'because',
  'we',
  'do',
  "n't",
  'see',
  'any',
  'grounds',
  'fo

In [6]:
sentences_in_dict = {}
sentences_in_dict['Word'] = sentences
sentences_in_dict

{'Word': [['EU',
   'rejects',
   'German',
   'call',
   'to',
   'boycott',
   'British',
   'lamb',
   '.'],
  ['Peter', 'Blackburn'],
  ['BRUSSELS', '1996-08-22'],
  ['The',
   'European',
   'Commission',
   'said',
   'on',
   'Thursday',
   'it',
   'disagreed',
   'with',
   'German',
   'advice',
   'to',
   'consumers',
   'to',
   'shun',
   'British',
   'lamb',
   'until',
   'scientists',
   'determine',
   'whether',
   'mad',
   'cow',
   'disease',
   'can',
   'be',
   'transmitted',
   'to',
   'sheep',
   '.'],
  ['Germany',
   "'s",
   'representative',
   'to',
   'the',
   'European',
   'Union',
   "'s",
   'veterinary',
   'committee',
   'Werner',
   'Zwingmann',
   'said',
   'on',
   'Wednesday',
   'consumers',
   'should',
   'buy',
   'sheepmeat',
   'from',
   'countries',
   'other',
   'than',
   'Britain',
   'until',
   'the',
   'scientific',
   'advice',
   'was',
   'clearer',
   '.'],
  ['"',
   'We',
   'do',
   "n't",
   'support',
   'any',
  

In [7]:
sentences_ner_tags

[['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'],
 ['B-PER', 'I-PER'],
 ['B-LOC', 'O'],
 ['O',
  'B-ORG',
  'I-ORG',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-LOC',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'I-ORG',
  'O',
  'O',
  'O',
  'B-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'O',
  'O',
  'O',
  'B-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'I-ORG',
 

In [8]:
ner_tags_in_dict = {}
ner_tags_in_dict['NER Tag'] = sentences_ner_tags
ner_tags_in_dict

{'NER Tag': [['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'],
  ['B-PER', 'I-PER'],
  ['B-LOC', 'O'],
  ['O',
   'B-ORG',
   'I-ORG',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-MISC',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-MISC',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O'],
  ['B-LOC',
   'O',
   'O',
   'O',
   'O',
   'B-ORG',
   'I-ORG',
   'O',
   'O',
   'O',
   'B-PER',
   'I-PER',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-LOC',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O'],
  ['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-ORG',
   'O',
   'O',
   'O',
   'B-PER',
   'I-PER',
   'I-PER',
   'I-PER',
   'O',
   'O',
   'O',
   'O',
   'O'],
  ['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O

In [9]:
sentences_in_dict.update(ner_tags_in_dict)
train_dict = sentences_in_dict

In [10]:
train_dict['NER Tag']

[['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'],
 ['B-PER', 'I-PER'],
 ['B-LOC', 'O'],
 ['O',
  'B-ORG',
  'I-ORG',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-LOC',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'I-ORG',
  'O',
  'O',
  'O',
  'B-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'O',
  'O',
  'O',
  'B-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'I-ORG',
 

In [11]:
tag2idx = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
idx2tag = {}

for key, value in tag2idx.items():
    idx2tag[value] = key

print(idx2tag)

{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC', 7: 'B-MISC', 8: 'I-MISC'}


In [12]:
def create_mappings(ner_tags, reordered_dict):
    """Pair word with corresponding index"""
    final_dict = {}
    per_sentence = []
    all_sentences = []

    # Iterate over the values_list
    for ner_tags_idx in range(len(ner_tags)):
        sentence_ner_tags = ner_tags[ner_tags_idx]
        # print(sentence_ner_tags)
        # assert(False)

        for sentence_ner_tags_idx in range(len(sentence_ner_tags)):
            ner_tag = sentence_ner_tags[sentence_ner_tags_idx]
            # print(ner_tag)

            # assert(False)

            for key, value in reordered_dict.items():
                # print("---", key, value)
                # assert(False)

                if ner_tag == value:
                    per_sentence.append(key)
                    # print("---", new_list) 
        # print(per_sentence)
        all_sentences.append(per_sentence)
        per_sentence = []
    # print(all_sentences)
    # assert(False)


    final_dict['NER Tag Idx'] = all_sentences

    return final_dict


In [13]:
ner_tags_idx = create_mappings(ner_tags_in_dict['NER Tag'], idx2tag)
ner_tags_idx

{'NER Tag Idx': [[3, 0, 7, 0, 0, 0, 7, 0, 0],
  [1, 2],
  [5, 0],
  [0,
   3,
   4,
   0,
   0,
   0,
   0,
   0,
   0,
   7,
   0,
   0,
   0,
   0,
   0,
   7,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [5,
   0,
   0,
   0,
   0,
   3,
   4,
   0,
   0,
   0,
   1,
   2,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   5,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   3,
   0,
   0,
   0,
   1,
   2,
   2,
   2,
   0,
   0,
   0,
   0,
   0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0],
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   3,
   0,
   0,
   1,
   2,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [1,
   0,
   7,
   0,
   0,
   0,
   0,
   5,

In [ ]:
# ner_tags_in_dict['NER Tag'][0]

In [14]:
train_dict.update(ner_tags_idx)

In [15]:
train_dict.keys()

dict_keys(['Word', 'NER Tag', 'NER Tag Idx'])

In [17]:
train_df = pd.DataFrame(train_dict)
train_df

,Word,NER Tag,NER Tag Idx
0,"[EU, rejects, German, call, to, boycott, Briti...","[B-ORG, O, B-MISC, O, O, O, B-MISC, O, O]","[3, 0, 7, 0, 0, 0, 7, 0, 0]"
1,"[Peter, Blackburn]","[B-PER, I-PER]","[1, 2]"
2,"[BRUSSELS, 1996-08-22]","[B-LOC, O]","[5, 0]"
3,"[The, European, Commission, said, on, Thursday...","[O, B-ORG, I-ORG, O, O, O, O, O, O, B-MISC, O,...","[0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, ..."
4,"[Germany, 's, representative, to, the, Europea...","[B-LOC, O, O, O, O, B-ORG, I-ORG, O, O, O, B-P...","[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, ..."
...,...,...,...
14982,"[Division, two]","[O, O]","[0, 0]"
14983,"[Plymouth, 2, Preston, 1]","[B-ORG, O, B-ORG, O]","[3, 0, 3, 0]"
14984,"[Division, three]","[O, O]","[0, 0]"
14985,"[Swansea, 1, Lincoln, 2]","[B-ORG, O, B-ORG, O]","[3, 0, 3, 0]"


In [18]:
train_df.loc[:, ['Word', 'NER Tag Idx']]

,Word,NER Tag Idx
0,"[EU, rejects, German, call, to, boycott, Briti...","[3, 0, 7, 0, 0, 0, 7, 0, 0]"
1,"[Peter, Blackburn]","[1, 2]"
2,"[BRUSSELS, 1996-08-22]","[5, 0]"
3,"[The, European, Commission, said, on, Thursday...","[0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, ..."
4,"[Germany, 's, representative, to, the, Europea...","[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, ..."
...,...,...
14982,"[Division, two]","[0, 0]"
14983,"[Plymouth, 2, Preston, 1]","[3, 0, 3, 0]"
14984,"[Division, three]","[0, 0]"
14985,"[Swansea, 1, Lincoln, 2]","[3, 0, 3, 0]"


In [20]:
train_dataset = Dataset.from_dict(train_df.loc[:, ['Word', 'NER Tag Idx']])
train_dataset

Dataset({
    features: ['Word', 'NER Tag Idx'],
    num_rows: 14987
})

### Load Glove Embeddings

In [ ]:
# input_file = 'glove.6B.100d.gz'
# output_file = 'glove.6B.100d.txt'

# with gzip.open(input_file, 'rb') as f_in:
#     with open(output_file, 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

In [21]:
import torch
import numpy as np
from tqdm import tqdm

def load_glove_embeddings(glove_file, embedding_dim, vocab_size=None):
    embeddings_index = {}
    vocabulary = []
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in tqdm(f, desc="Loading GloVe embeddings", total=vocab_size):
            values = line.split()
            word = values[0]
            vocabulary.append(word)
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    # Create a matrix to hold the embeddings
    embedding_matrix = torch.randn((vocab_size, embedding_dim))
    for i, (word, embedding_vector) in tqdm(enumerate(embeddings_index.items()), desc="Creating embedding matrix"):
        if i < vocab_size:
            embedding_matrix[i] = torch.from_numpy(embedding_vector)

    return embedding_matrix, vocabulary

# Example usage
glove_file = 'glove.6B.100d.txt'
embedding_dim = 100
vocab_size = 10000  # Adjust this according to your vocabulary size
embedding_matrix, vocabulary = load_glove_embeddings(glove_file, embedding_dim, vocab_size)

# Now, embedding_matrix contains the GloVe embeddings loaded into PyTorch tensors
# vocabulary contains the list of words in the GloVe embeddings


Loading GloVe embeddings: 400000it [00:10, 36887.85it/s]                         
Creating embedding matrix: 400000it [00:00, 1733597.17it/s]


In [22]:
embedding_matrix

tensor([[-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        [-0.1077,  0.1105,  0.5981,  ..., -0.8316,  0.4529,  0.0826],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [ 0.0565,  1.1955, -0.1534,  ...,  0.1654,  0.4140, -0.2897],
        [-1.1035,  0.6795, -0.1155,  ..., -1.1947,  0.2672, -0.5012],
        [-0.0837, -0.3817, -0.2315,  ...,  0.3612, -0.4694, -0.2646]])

In [96]:
glove_embeddings = embedding_matrix.numpy()
glove_embeddings

array([[-0.038194, -0.24487 ,  0.72812 , ..., -0.1459  ,  0.8278  ,
         0.27062 ],
       [-0.10767 ,  0.11053 ,  0.59812 , ..., -0.83155 ,  0.45293 ,
         0.082577],
       [-0.33979 ,  0.20941 ,  0.46348 , ..., -0.23394 ,  0.47298 ,
        -0.028803],
       ...,
       [ 0.05649 ,  1.1955  , -0.15339 , ...,  0.16537 ,  0.41398 ,
        -0.28969 ],
       [-1.1035  ,  0.67954 , -0.11548 , ..., -1.1947  ,  0.26718 ,
        -0.50118 ],
       [-0.083695, -0.38169 , -0.23151 , ...,  0.36124 , -0.46944 ,
        -0.26457 ]], dtype=float32)

In [23]:
vocabulary[:7]
# vocab_npa

['the', ',', '.', 'of', 'to', 'and', 'in']

In [24]:
def add_special_tokens_to_list(word_list):
    """
    Adds special tokens 'PAD' and 'UNK' to the beginning of the list.
    
    Parameters:
    word_list (list): The original list of words.
    
    Returns:
    list: The list with special tokens added.
    """
    # Add 'PAD' and 'UNK' tokens to the beginning of the list
    word_list = ['PAD', 'UNK'] + word_list
    
    return np.array(word_list)


In [25]:
word_list_with_special_tokens = add_special_tokens_to_list(vocabulary)
word_list_with_special_tokens

array(['PAD', 'UNK', 'the', ..., 'rolonda', 'zsombor', 'sandberger'],
      dtype='<U68')

In [26]:
import torch

def add_special_embeddings(embs_tensor):
    """
    Adds special embeddings for 'PAD' and 'UNK' tokens to the beginning of the tensor.
    
    Parameters:
    embs_tensor (torch.Tensor): The original tensor of word embeddings.
    
    Returns:
    torch.Tensor: The tensor with special embeddings added.
    """
    # Create a tensor filled with zeros for the 'PAD' token
    pad_emb_tensor = torch.zeros((1, embs_tensor.shape[1]), dtype=torch.float32)
    
    # Calculate the mean of all word embeddings for the 'UNK' token
    unk_emb_tensor = torch.mean(embs_tensor, dim=0, keepdim=True)
    
    # Concatenate 'PAD', 'UNK', and original embeddings tensors along the first dimension
    embs_tensor_with_special = torch.cat((pad_emb_tensor, unk_emb_tensor, embs_tensor), dim=0)
    
    return embs_tensor_with_special


In [27]:
# Example usage:
embs_tensor_with_special = add_special_embeddings(embedding_matrix)
embs_tensor_with_special.shape

torch.Size([10002, 100])

In [91]:
def build_labels(dataset, pad_token="PAD", unk_token="UNK"):
  labels = []


  for row in tqdm(dataset, total=len(dataset)):
    labels.extend(row['NER Tag Idx'])

  labels = list(set(labels))

  return labels

In [92]:
labels = build_labels(train_dataset)

  0%|          | 0/14987 [00:00<?, ?it/s]

100%|██████████| 14987/14987 [00:01<00:00, 14703.80it/s]


In [93]:
labels

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [46]:
def build_vocab_glove(words):
    """Build vocabulary for GloVe embeddings.

    Parameters
    ----------
    words: `list`:
        List of vocabulary terms.

    Returns
    -------
    word_idx, idx_word: `tuple`
        A tuple containing two dictionaries mapping words to their corresponding indices and mapping indices to their corresponding words.
    """
    word_at_idx = {}

    for word_idx, word in enumerate(words):
        # print(word, word_idx)
        
        word_at_idx[word] = word_idx
        # assert(False)

    idx_at_word = {}

    for word, word_idx in word_at_idx.items():
        # print(word, idx)

        idx_at_word[word_idx] = word
        
    return word_at_idx, idx_at_word


In [47]:
word2idx, idx2word = build_vocab_glove(word_list_with_special_tokens)

In [30]:
word2idx

{'PAD': 0,
 'UNK': 1,
 'the': 2,
 ',': 3,
 '.': 4,
 'of': 5,
 'to': 6,
 'and': 7,
 'in': 8,
 'a': 9,
 '"': 10,
 "'s": 11,
 'for': 12,
 '-': 13,
 'that': 14,
 'on': 15,
 'is': 16,
 'was': 17,
 'said': 18,
 'with': 19,
 'he': 20,
 'as': 21,
 'it': 22,
 'by': 23,
 'at': 24,
 '(': 25,
 ')': 26,
 'from': 27,
 'his': 28,
 "''": 29,
 '``': 30,
 'an': 31,
 'be': 32,
 'has': 33,
 'are': 34,
 'have': 35,
 'but': 36,
 'were': 37,
 'not': 38,
 'this': 39,
 'who': 40,
 'they': 41,
 'had': 42,
 'i': 43,
 'which': 44,
 'will': 45,
 'their': 46,
 ':': 47,
 'or': 48,
 'its': 49,
 'one': 50,
 'after': 51,
 'new': 52,
 'been': 53,
 'also': 54,
 'we': 55,
 'would': 56,
 'two': 57,
 'more': 58,
 "'": 59,
 'first': 60,
 'about': 61,
 'up': 62,
 'when': 63,
 'year': 64,
 'there': 65,
 'all': 66,
 '--': 67,
 'out': 68,
 'she': 69,
 'other': 70,
 'people': 71,
 "n't": 72,
 'her': 73,
 'percent': 74,
 'than': 75,
 'over': 76,
 'into': 77,
 'last': 78,
 'some': 79,
 'government': 80,
 'time': 81,
 '$': 82,
 'you

In [48]:
idx2word

{0: 'PAD',
 1: 'UNK',
 2: 'the',
 3: ',',
 4: '.',
 5: 'of',
 6: 'to',
 7: 'and',
 8: 'in',
 9: 'a',
 10: '"',
 11: "'s",
 12: 'for',
 13: '-',
 14: 'that',
 15: 'on',
 16: 'is',
 17: 'was',
 18: 'said',
 19: 'with',
 20: 'he',
 21: 'as',
 22: 'it',
 23: 'by',
 24: 'at',
 25: '(',
 26: ')',
 27: 'from',
 28: 'his',
 29: "''",
 30: '``',
 31: 'an',
 32: 'be',
 33: 'has',
 34: 'are',
 35: 'have',
 36: 'but',
 37: 'were',
 38: 'not',
 39: 'this',
 40: 'who',
 41: 'they',
 42: 'had',
 43: 'i',
 44: 'which',
 45: 'will',
 46: 'their',
 47: ':',
 48: 'or',
 49: 'its',
 50: 'one',
 51: 'after',
 52: 'new',
 53: 'been',
 54: 'also',
 55: 'we',
 56: 'would',
 57: 'two',
 58: 'more',
 59: "'",
 60: 'first',
 61: 'about',
 62: 'up',
 63: 'when',
 64: 'year',
 65: 'there',
 66: 'all',
 67: '--',
 68: 'out',
 69: 'she',
 70: 'other',
 71: 'people',
 72: "n't",
 73: 'her',
 74: 'percent',
 75: 'than',
 76: 'over',
 77: 'into',
 78: 'last',
 79: 'some',
 80: 'government',
 81: 'time',
 82: '$',
 83: 

In [57]:
# Define the conversion function
def convert_text_to_input_ids_glove(row):
    """
    Convert tokenized text into input IDs using GloVe embeddings.
    
    Args:
        row (dict): A dictionary containing the tokens for a single data point.
        
    Returns:
        dict: A dictionary containing the input IDs and capitalization vector for the tokens.
            - 'ids_of_inputs' (list): List of input IDs corresponding to each token.
            - 'capital_words_vector' (list): Binary vector indicating capitalization of each token.
    
    Note:
        This function requires the 'word2idx' dictionary to map tokens to their GloVe indices.
        It assumes that each token is available in the 'word2idx' dictionary.
    """
    capital_words_vector = []  # Initialize list to store capitalization vector
    ids_of_inputs = []  # Initialize list to store input IDs
    for Word in row['Word']:  # Iterate through each word in the row
        # Check if any character in the word is uppercase
        if any(x.isupper() for x in Word):
            capital_words_vector.append(1)  # Append 1 if uppercase character is found
        else:
            capital_words_vector.append(0)  # Append 0 if no uppercase character is found
        
        Word_lower = Word.lower()  # Convert word to lowercase
        # Check if lowercase word is in word2idx dictionary
        if Word_lower in word2idx:
            ids_of_inputs.append(word2idx[Word_lower])  # Append corresponding index from word2idx
        else:
            ids_of_inputs.append(1)  # Append index 1 as default if word not found in word2idx
    
    return {
        'ids_of_inputs': ids_of_inputs,  # Return input IDs
        'capital_words_vector': capital_words_vector  # Return capitalization vector
    }

In [62]:
# Assign the conversion function to a variable
convert_function = convert_text_to_input_ids_glove

# Create an empty list to store converted data points
converted_data = []

# Iterate through each row in the dataset and apply the conversion function
for row in train_dataset:
    converted_data.append(convert_function(row))

# Now 'converted_data' contains the converted dataset


In [121]:
def convert_dataset_to_glove(dataset, convert_function):
    # Create an empty list to store converted data points
    converted_data = []

    # Iterate through each row in the dataset and apply the conversion function
    for row in dataset:
        converted_row = {}
        converted_row['Word'] = row['Word']
        converted_row['NER Tag Idx'] = row['NER Tag Idx']
        converted_result = convert_function(row)
        converted_row['ids_of_inputs'] = converted_result['ids_of_inputs']
        converted_row['capital_words_vector'] = converted_result['capital_words_vector']
        converted_data.append(converted_row)

    # Now 'converted_data' contains the converted dataset

    # Create the converted_data_dict
    converted_data_dict = {}
    for key in converted_data[0]:
        converted_data_dict[key] = []
        for row in converted_data:
            converted_data_dict[key].append(row[key])

    # Create the converted_dataset
    converted_dataset = Dataset.from_dict(converted_data_dict)

    return converted_dataset

# Usage example:
train_dataset_glove = convert_dataset_to_glove(train_dataset, convert_text_to_input_ids_glove)


In [122]:
train_dataset_glove

Dataset({
    features: ['Word', 'NER Tag Idx', 'ids_of_inputs', 'capital_words_vector'],
    num_rows: 14987
})

In [125]:
train_dataset_glove.set_format(
    type='torch',
    columns=['Word', 'NER Tag Idx', 'ids_of_inputs', 'capital_words_vector']
)


In [126]:
print(train_dataset_glove[0])

{'Word': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'NER Tag Idx': tensor([3, 0, 7, 0, 0, 0, 7, 0, 0]), 'ids_of_inputs': tensor([  646,  7580,   516,   582,     6,  5262,   299, 10240,     4]), 'capital_words_vector': tensor([1, 0, 1, 0, 0, 0, 1, 0, 0])}


In [127]:
train_dataset_glove.set_format(
    type='torch',
    columns=['NER Tag Idx', 'ids_of_inputs', 'capital_words_vector']
)
print(train_dataset_glove[0])

{'NER Tag Idx': tensor([3, 0, 7, 0, 0, 0, 7, 0, 0]), 'ids_of_inputs': tensor([  646,  7580,   516,   582,     6,  5262,   299, 10240,     4]), 'capital_words_vector': tensor([1, 0, 1, 0, 0, 0, 1, 0, 0])}


In [128]:
from torch.nn.utils.rnn import pad_sequence

def pad_sequences(batch):
    """
    Custom collate function to pad sequences in a batch.

    Combines individual samples into batches, padding sequences to the same length.
    
    Args:
        batch (list): A list of dictionaries, each containing ids_of_inputs, capital_words_vector, and NER Tag Idx.

    Returns:
        dict: A dictionary containing padded sequences of ids_of_inputs, capital_words_vector, and NER Tag Idx.
    """
    # Initialize lists to store ids_of_inputs, capital_words_vector, and NER Tag Idx
    ids_of_inputs_list = []
    capital_words_vector_list = []
    ner_tag_idx_list = []

    # Iterate through each item in the batch
    for x in batch:
        # Get ids_of_inputs, capital_words_vector, and NER Tag Idx from the current item
        ids_of_inputs = x['ids_of_inputs']
        capital_words_vector = x['capital_words_vector']
        ner_tag_idx = x['NER Tag Idx']

        # Append ids_of_inputs, capital_words_vector, and NER Tag Idx to respective lists
        ids_of_inputs_list.append(ids_of_inputs)
        capital_words_vector_list.append(capital_words_vector)
        ner_tag_idx_list.append(ner_tag_idx)

    # Pad sequences with zeros, ensuring they are padded to the length of the longest sequence in the batch
    padded_ids_of_inputs = pad_sequence(ids_of_inputs_list, batch_first=True)
    padded_capital_words_vector = pad_sequence(capital_words_vector_list, batch_first=True)
    # Pad NER Tag Idx with -1, ensuring they are padded to the length of the longest sequence in the batch
    padded_ner_tag_idx = pad_sequence(ner_tag_idx_list, batch_first=True, padding_value=-1)

    # Create a dictionary containing padded sequences of ids_of_inputs, capital_words_vector, and NER Tag Idx
    padded_data = {
        "ids_of_inputs": padded_ids_of_inputs,
        "capital_words_vector": padded_capital_words_vector,
        "NER Tag Idx": padded_ner_tag_idx
    }

    # Return the dictionary
    return padded_data


# Outline of Tasks

1. Simple Bidirectional LSTM model
2. Using GloVe word embeddings
3. BONUS: LSTM-CNN model


# 1. Simple Bidirectional LSTM model

- LSTM embedding should be per sentence
- I currently have each embedding per word

### Define Hyper-parameters

In [129]:
EMBEDDING_DIM = 100
LSTM_LAYERS = 1

HIDDEN_DIM = 256
DROPOUT = 0.33

OUTPUT_DIM = 128

TRAIN_BATCH_SIZE = 64

train_params = {
    'batch_size': TRAIN_BATCH_SIZE,
    'shuffle': True,
    'num_workers': 0
}

### Create the BLSTM

In [130]:
train_dataloader_glove = torch.utils.data.DataLoader(train_dataset_glove, collate_fn=pad_sequences, **train_params)

In [131]:
train_dataloader_glove

In [132]:
class NERTaggerGlove(nn.Module):
  def __init__(self, embed_dim, hidden_dim, vocab_size, target_vocab_size, pretrained_embeddings, dropout=0.33):
    super(NERTaggerGlove, self).__init__()

    self.embed_dim = embed_dim
    self.hidden_dim = hidden_dim
    self.linear_dim = 128
    self.target_size = target_vocab_size
    self.vocab_size = vocab_size
    self.dropout = dropout

    # Layers
    self.embedding = nn.Embedding.from_pretrained(torch.from_numpy(pretrained_embeddings).float())
    self.lstm = nn.LSTM(self.embed_dim+1, self.hidden_dim, bidirectional=True, batch_first=True, num_layers=1)
    self.dropout = nn.Dropout(self.dropout)
    self.linear = nn.Linear(self.hidden_dim*2, self.linear_dim)
    self.elu = nn.ELU()
    self.classifier = nn.Linear(self.linear_dim, self.target_size)

  def forward(self, input, capital_vectors, labels=None):
    embeddings = self.embedding(input)
    capital_vectors = capital_vectors.unsqueeze(2)

    # concatenating captial vectors at the end of embeddings
    concatenated_input = torch.cat([embeddings, capital_vectors], dim=2)

    out, hidden = self.lstm(concatenated_input)
    out = self.dropout(out)
    out = self.linear(out)
    out = self.elu(out)
    out = self.classifier(out)
    loss = None
    if labels is not None:
      loss = nn.functional.cross_entropy(out.permute(0, 2, 1), labels, ignore_index=-1)
    return out, loss


### TRAIN THE BLSTM

In [133]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: ', device.type)

Device:  cpu


In [134]:
model_glove = NERTaggerGlove(embed_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, vocab_size=len(word2idx), target_vocab_size=len(labels), pretrained_embeddings=glove_embeddings, dropout=DROPOUT)
model_glove.to(device)


NERTaggerGlove(
  (embedding): Embedding(10000, 100)
  (lstm): LSTM(101, 256, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.33, inplace=False)
  (linear): Linear(in_features=512, out_features=128, bias=True)
  (elu): ELU(alpha=1.0)
  (classifier): Linear(in_features=128, out_features=9, bias=True)
)

In [135]:
LEARNING_RATE = 1
LOSS_FUNCTION = nn.CrossEntropyLoss()
OPTIMIZER = optim.SGD(model_glove.parameters(), lr=LEARNING_RATE)

In [136]:
def train_glove(model, train_dataloader, target_vocab_size, val_dataloader, epochs=1):
  train_losses = []
  best_val_score = 0
  for epoch in range(epochs):
    train_loss = 0.0

    model.train()
    for batch in train_dataloader:
      OPTIMIZER.zero_grad()

      sentences = batch['ids_of_inputs'].to(device, torch.long)
      capital_vectors = batch['capital_words_vector'].to(device, torch.long)
      labels = batch['NER Tag Idx'].to(device, torch.long)

      output, loss = model(sentences, capital_vectors, labels)
      loss.backward()
      OPTIMIZER.step()

      train_loss += loss.item()

    train_loss = train_loss/len(train_dataloader)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))

    train_losses.append(train_loss)

In [137]:
train_losses = train_glove(model_glove, train_dataloader_glove, len(labels), "val_dataloader_glove", epochs=2)

IndexError: index out of range in self